In [1]:
import pandas as pd
import numpy as np
import os
import re
import contractions
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import opinion_lexicon

import statsmodels.formula.api as smf
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

# event_study

In [2]:
import pandas_datareader
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
estimation_startdate = datetime(2021,8,24)
estimation_enddate = datetime(2022,2,20)

# 'FTSE 100', 'NASDAQ-100', 'S&P 500', 'CAC 40', 'DAX',
#           'Nikkei 225','Hang Seng Index',  'Shanghai Composite Index', 'S&P/ASX 200', 'S&P BSE SENSEX ',
#           'S&P/TSX Composite Index', 'STI Index', 'ESTX 50 PR.EUR','Russell 2000', 'FTSE MIB Index',
#           'S&P/ASX 50', 'Nifty 50', 'OMX Stockholm 30', 'PSI 20', 'Dow Jones Industrial Average',
#           'iShares MSCI World Index ETF', 'KOSPI Composite Index', 'OMX Copenhagen 25', 'TSEC weighted index', 
#            'S&P/NZX 50 INDEX GROSS', 'STXE 600 PR.EUR','BIST 100',
#            'iShares MSCI World Small Cap UCITS ETF ', 'MSCI AC Asia Pacific EX Japan Index FAM Fund Class I EUR Accumulation ',
#             'Vanguard FTSE Developed Europe ex-U.K. Equity Index Fund GBP Acc'

tickers = ['IMOEX.ME','^FTSE','^FCHI','DAX', 'ENY=F', '^HSI', '000001.SS','^AXJO', '^BSESN', 
           '^GSPTSE', '^STI', '^STOXX50E', '^RUT', 'FTSEMIB.MI', '^AFLI', '^NSEI','^OMX',
           'PSI20.LS', '^DJI', 'XWD.TO','^KS11','^OMXC25','^TWII','^NZ50', '^STOXX',
           'XU100.IS', 'WSML.L', '0P0001GY56.F', 'IEUR',  '0P0000KSP8.L']
marketPrices = pdr.get_data_yahoo(tickers, start = estimation_startdate, end = estimation_enddate)["Adj Close"]

[*********************100%***********************]  30 of 30 completed


In [4]:
marketPrices.head(10)

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-24,3514.470947,34365.121094,123.935997,31.831852,27770.0,26028.0,55.264896,3888.870117,5361.770020,7.329,...,3138.300049,16624.599609,13071.860352,2367.050049,2001.260010,2230.909912,3107.620117,471.790009,4178.080078,16818.730469
2021-08-25,3540.379883,34283.878906,124.518997,31.761326,27800.0,26060.0,55.367893,3886.639893,5350.609863,7.391,...,3146.810059,16634.650391,13173.480469,2376.159912,1987.839966,2239.270020,3107.489990,471.839996,4181.120117,17045.859375
2021-08-26,3501.664062,34237.929688,123.634003,31.619324,27650.0,25862.0,55.058891,3851.570068,5330.779785,7.352,...,3128.530029,16636.900391,13051.620117,2371.699951,1988.550049,2213.979980,3109.419922,470.339996,4169.870117,17066.960938
2021-08-27,3522.156982,34422.308594,123.695000,31.860062,27860.0,26007.0,55.648808,3887.379883,5325.870117,7.439,...,3133.899902,16705.199219,13059.790039,2373.139893,1983.579956,2277.149902,3080.770020,472.339996,4190.979980,17209.929688
2021-08-30,3528.150879,NaN,124.712997,31.869467,27700.0,26024.0,55.676903,3928.500000,5340.410156,NaN,...,3144.189941,16931.050781,13180.580078,2372.810059,1993.130005,2265.989990,3102.110107,472.679993,4198.799805,17396.519531
2021-08-31,3543.939941,34323.269531,126.615997,31.850655,28070.0,26009.0,55.527073,3918.959961,5417.080078,7.456,...,3199.270020,17132.199219,13218.830078,2351.250000,1975.239990,2273.770020,3055.050049,470.880005,4196.410156,17490.289062
2021-09-01,3567.100098,34601.949219,126.632004,31.925890,28540.0,26182.0,55.892265,3971.020020,5472.529785,7.494,...,3207.020020,17076.250000,13243.490234,2368.669922,1988.310059,2287.060059,3087.840088,473.119995,4227.270020,17473.990234
2021-09-02,3597.040039,34620.960938,126.676003,32.019928,28610.0,26233.0,56.219994,3990.770020,5505.549805,7.573,...,3175.850098,17234.150391,13280.469727,2388.399902,2012.359985,2304.020020,3088.840088,474.600006,4232.100098,17319.759766
2021-09-03,3581.729980,34327.640625,126.999001,32.029331,29600.0,26065.0,55.976536,4001.699951,5488.359863,7.555,...,3201.060059,17323.599609,13288.870117,2369.729980,2003.869995,2292.050049,3083.850098,471.929993,4201.979980,17516.919922


In [5]:
marketReturns  = marketPrices.pct_change(1).dropna()
#marketReturns = np.where(int(marketReturns) == 0, )
marketReturns.head()

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-25,0.007372,-0.002364,0.004704,-0.002216,0.001080,0.001229,0.001864,-0.000573,-0.002081,0.008460,...,0.002712,0.000605,0.007774,0.003849,-0.006706,0.003747,-0.000042,0.000106,0.000728,0.013505
2021-08-26,-0.010935,-0.001340,-0.007107,-0.004471,-0.005396,-0.007598,-0.005581,-0.009023,-0.003706,-0.005277,...,-0.005809,0.000135,-0.009250,-0.001877,0.000357,-0.011294,0.000621,-0.003179,-0.002691,0.001238
2021-08-27,0.005852,0.005385,0.000493,0.007614,0.007595,0.005607,0.010714,0.009297,-0.000921,0.011833,...,0.001716,0.004105,0.000626,0.000607,-0.002499,0.028532,-0.009214,0.004252,0.005062,0.008377
2021-08-30,0.001702,0.000000,0.008230,0.000295,-0.005743,0.000654,0.000505,0.010578,0.002730,0.000000,...,0.003283,0.013520,0.009249,-0.000139,0.004815,-0.004901,0.006927,0.000720,0.001866,0.010842
2021-08-31,0.004475,-0.002877,0.015259,-0.000590,0.013357,-0.000576,-0.002691,-0.002428,0.014357,0.002285,...,0.017518,0.011880,0.002902,-0.009086,-0.008976,0.003433,-0.015170,-0.003808,-0.000569,0.005390


In [6]:
marketReturns["whole market"] = marketReturns.mean(axis = 1)
marketReturns.head(10)

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,whole market
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-25,0.007372,-0.002364,0.004704,-0.002216,0.001080,0.001229,0.001864,-0.000573,-0.002081,0.008460,...,0.000605,0.007774,0.003849,-0.006706,0.003747,-0.000042,0.000106,0.000728,0.013505,0.001642
2021-08-26,-0.010935,-0.001340,-0.007107,-0.004471,-0.005396,-0.007598,-0.005581,-0.009023,-0.003706,-0.005277,...,0.000135,-0.009250,-0.001877,0.000357,-0.011294,0.000621,-0.003179,-0.002691,0.001238,-0.004301
2021-08-27,0.005852,0.005385,0.000493,0.007614,0.007595,0.005607,0.010714,0.009297,-0.000921,0.011833,...,0.004105,0.000626,0.000607,-0.002499,0.028532,-0.009214,0.004252,0.005062,0.008377,0.004566
2021-08-30,0.001702,0.000000,0.008230,0.000295,-0.005743,0.000654,0.000505,0.010578,0.002730,0.000000,...,0.013520,0.009249,-0.000139,0.004815,-0.004901,0.006927,0.000720,0.001866,0.010842,0.002830
2021-08-31,0.004475,-0.002877,0.015259,-0.000590,0.013357,-0.000576,-0.002691,-0.002428,0.014357,0.002285,...,0.011880,0.002902,-0.009086,-0.008976,0.003433,-0.015170,-0.003808,-0.000569,0.005390,0.002649
2021-09-01,0.006535,0.008119,0.000126,0.002362,0.016744,0.006652,0.006577,0.013284,0.010236,0.005097,...,-0.003266,0.001866,0.007409,0.006617,0.005845,0.010733,0.004757,0.007354,-0.000932,0.004812
2021-09-02,0.008393,0.000549,0.000347,0.002946,0.002453,0.001948,0.005864,0.004974,0.006034,0.010542,...,0.009247,0.002792,0.008330,0.012096,0.007416,0.000324,0.003128,0.001143,-0.008826,0.002644
2021-09-03,-0.004256,-0.008472,0.002550,0.000294,0.034603,-0.006404,-0.004330,0.002739,-0.003122,-0.002377,...,0.005190,0.000633,-0.007817,-0.004219,-0.005195,-0.001615,-0.005626,-0.007117,0.011384,-0.000360
2021-09-06,0.011204,0.009107,0.007937,0.000000,0.000000,0.007596,0.000000,0.006635,-0.001838,0.004633,...,0.003129,0.000840,0.010351,0.006612,0.000000,0.005587,0.006908,0.010507,-0.001234,0.004054


In [7]:
Beta = {}
var = marketReturns["whole market"].var()
for col in marketReturns:
    cov = marketReturns[col].cov(marketReturns["whole market"])
    Beta[col] = cov/var
Beta

{'000001.SS': 0.29851268179896734,
 '0P0000KSP8.L': 1.1772181130328405,
 '0P0001GY56.F': 0.6843888900398365,
 'DAX': 1.261039144912299,
 'ENY=F': 1.1545362860839543,
 'FTSEMIB.MI': 1.4821463166357642,
 'IEUR': 1.1066345636314057,
 'IMOEX.ME': 1.6266139887671287,
 'PSI20.LS': 1.0456595930098505,
 'WSML.L': 1.5766261762869056,
 'XU100.IS': 1.0425784547015824,
 'XWD.TO': 0.7239029840785247,
 '^AFLI': 0.5215066599707564,
 '^AXJO': 0.6034003863870331,
 '^BSESN': 0.9930015169024723,
 '^DJI': 0.8538745470268004,
 '^FCHI': 1.4195263684457688,
 '^FTSE': 0.9249666982090771,
 '^GSPTSE': 0.7962004221679618,
 '^HSI': 0.8667495574829337,
 '^KS11': 0.6475875879488111,
 '^NSEI': 1.0004547237733499,
 '^NZ50': 0.41855767866734944,
 '^OMX': 1.4281015674220185,
 '^OMXC25': 1.427763207668167,
 '^RUT': 1.1715418426573958,
 '^STI': 0.40720081973095584,
 '^STOXX': 1.2876845285350194,
 '^STOXX50E': 1.532774146098581,
 '^TWII': 0.519250547926486,
 'whole market': 1.0}

In [8]:
event_startdate = datetime(2022,2,18)
event_enddate = datetime(2022,3,2)
eventPrices = pdr.get_data_yahoo(tickers, start = event_startdate, end = event_enddate)["Adj Close"]
eventPrices

[*********************100%***********************]  30 of 30 completed


,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-18,3490.760010,32164.330078,123.538002,29.034582,26850.0,26507.0,52.391735,3393.310059,5632.200195,6.847,...,2744.520020,17276.300781,12141.889648,2197.719971,1684.910034,2009.329956,3428.899902,460.809998,4074.280029,18232.349609
2022-02-21,3490.610107,31566.750000,123.113998,NaN,NaN,26050.0,NaN,3036.879883,5493.490234,6.765,...,2743.800049,17206.650391,12156.339844,2159.300049,1659.369995,NaN,3436.360107,454.809998,3985.709961,18221.490234
2022-02-22,3457.149902,31613.890625,121.375999,28.217510,26660.0,26044.0,51.584690,3084.739990,5464.919922,6.793,...,2706.790039,17092.199219,12114.629883,2157.919922,1661.310059,1980.170044,3400.580078,455.119995,3985.469971,17969.289062
2022-02-23,3489.149902,31569.140625,121.794998,27.856478,26330.0,25955.0,51.166920,NaN,5430.930176,6.717,...,2719.530029,17063.250000,12134.419922,2142.030029,1651.420044,1944.089966,3393.000000,453.859985,3973.409912,18055.730469
2022-02-24,3429.959961,30680.599609,119.671997,27.305428,26260.0,24878.0,50.207954,2058.120117,5348.279785,6.564,...,2648.800049,16247.950195,11967.019531,2078.060059,1657.300049,1996.010010,3276.060059,438.959991,3829.290039,17594.550781
2022-02-25,3451.409912,31691.130859,119.325996,28.179506,26980.0,25773.0,51.679634,2470.479980,5495.209961,6.791,...,2676.760010,16658.400391,11923.379883,2141.520020,1713.150024,2040.930054,3294.469971,453.529999,3970.689941,17652.179688
2022-02-28,3462.310059,31634.900391,NaN,27.252224,26650.0,25416.0,50.511791,NaN,5563.140137,6.906,...,2699.179932,16793.900391,12096.679688,2134.070068,1764.810059,2048.090088,3242.239990,453.109985,3924.229980,NaN
2022-03-01,3488.830078,30702.449219,NaN,26.288839,26430.0,24364.0,49.154045,NaN,5498.270020,6.767,...,NaN,NaN,12155.530273,2084.929932,1752.020020,2008.510010,3278.629883,442.369995,3765.850098,17898.250000


In [9]:
eventReturns = eventPrices.pct_change(1).dropna()
eventReturns["whole market"] = eventReturns.mean(axis = 1)
eventReturns

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,whole market
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-21,-0.000043,-0.018579,-0.003432,0.000000,0.000000,-0.017241,0.000000,-0.105039,-0.024628,-0.011976,...,-0.004032,0.001190,-0.017482,-0.015158,0.000000,0.002176,-0.013021,-0.021739,-0.000596,-0.009206
2022-02-22,-0.009586,0.001493,-0.014117,-0.028141,-0.007076,-0.000230,-0.015404,0.015760,-0.005201,0.004139,...,-0.006652,-0.003431,-0.000639,0.001169,-0.014512,-0.010412,0.000682,-0.000060,-0.013841,-0.007000
2022-02-23,0.009256,-0.001416,0.003452,-0.012795,-0.012378,-0.003417,-0.008099,0.000000,-0.006220,-0.011188,...,-0.001694,0.001634,-0.007364,-0.005953,-0.018221,-0.002229,-0.002769,-0.003026,0.004811,-0.003158
2022-02-24,-0.016964,-0.028146,-0.017431,-0.019782,-0.002659,-0.041495,-0.018742,-0.332806,-0.015218,-0.022778,...,-0.047781,-0.013796,-0.029864,0.003561,0.026707,-0.034465,-0.032829,-0.036271,-0.025542,-0.033841
2022-02-25,0.006254,0.032937,-0.002891,0.032011,0.027418,0.035976,0.029312,0.200358,0.027472,0.034583,...,0.025262,-0.003647,0.030538,0.033699,0.022505,0.005620,0.033192,0.036926,0.003275,0.027440
2022-02-28,0.003158,-0.001774,0.000000,-0.032906,-0.012231,-0.013852,-0.022598,0.000000,0.012362,0.016934,...,0.008134,0.014534,-0.003479,0.030155,0.003508,-0.015854,-0.000926,-0.011701,0.000000,-0.001011
2022-03-01,0.007660,-0.029475,0.000000,-0.035351,-0.008255,-0.041391,-0.026880,0.000000,-0.011661,-0.020127,...,0.000000,0.004865,-0.023026,-0.007247,-0.019325,0.011224,-0.023703,-0.040359,0.013940,-0.010599


In [10]:
non_risk_insterest_rate = 0.0384

abnormalReturns = pd.DataFrame()

for col in marketReturns:
    abnormalReturns[col] = non_risk_insterest_rate + Beta[col]*(eventReturns[col] - non_risk_insterest_rate) - eventReturns[col]

abnormalReturns

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,whole market
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-21,0.026967,-0.010098,0.013203,-0.010024,-0.005934,-0.026827,-0.004095,-0.089881,-0.002878,-0.029048,...,-0.000019,0.021635,-0.023923,-0.022910,-0.006587,0.021474,-0.014793,-0.032040,0.018747,3.469447e-18
2022-02-22,0.033661,-0.006541,0.016575,-0.017370,-0.007028,-0.018625,-0.005737,-0.014187,-0.001991,-0.019756,...,-0.000020,0.024322,-0.016713,-0.015926,-0.009077,0.028936,-0.010851,-0.020491,0.025115,1.734723e-18
2022-02-23,0.020444,-0.007056,0.011030,-0.013364,-0.007847,-0.020162,-0.004958,-0.024062,-0.002037,-0.028594,...,-0.000018,0.021378,-0.019591,-0.018973,-0.009713,0.024085,-0.011844,-0.022071,0.016148,-2.168404e-18
2022-02-24,0.038837,-0.011793,0.017621,-0.015188,-0.006345,-0.038521,-0.006093,-0.232603,-0.002448,-0.035277,...,-0.000039,0.030349,-0.029224,-0.014903,-0.002006,0.043194,-0.020492,-0.039783,0.030740,6.938894e-18
2022-02-25,0.022550,-0.000968,0.013032,-0.001668,-0.001697,-0.001169,-0.000969,0.101485,-0.000499,-0.002201,...,-0.000006,0.024448,-0.003366,-0.002011,-0.002727,0.019432,-0.001498,-0.000785,0.016886,0.000000e+00
2022-02-28,0.024722,-0.007120,0.012119,-0.018614,-0.007824,-0.025193,-0.006504,-0.024062,-0.001189,-0.012378,...,-0.000014,0.013876,-0.017928,-0.003527,-0.005985,0.032162,-0.011314,-0.026692,0.018461,-2.385245e-18
2022-03-01,0.021564,-0.012029,0.012119,-0.019252,-0.007210,-0.038471,-0.006961,-0.024062,-0.002286,-0.033748,...,-0.000017,0.019499,-0.026297,-0.019526,-0.009902,0.016110,-0.017866,-0.041961,0.011759,-1.734723e-18


In [11]:
all_AR = pd.DataFrame(abnormalReturns.mean(axis = 1)/len(tickers))
all_AR.columns = ['AAR']
all_AR

,AAR
Date,
2022-02-21,-0.000134
2022-02-22,0.000051
2022-02-23,-0.000038
2022-02-24,-0.000229
2022-02-25,0.000259
2022-02-28,-0.000019
2022-03-01,-0.000139


In [12]:
CAR_list = [all_AR.iloc[0]["AAR"]]
for i in range(1, len(all_AR)):
    CAR_list.append(all_AR.iloc[i-1]["AAR"]+all_AR.iloc[i]["AAR"])
all_AR["CAR"] = CAR_list
all_AR

,AAR,CAR
Date,,
2022-02-21,-0.000134,-0.000134
2022-02-22,0.000051,-0.000083
2022-02-23,-0.000038,0.000013
2022-02-24,-0.000229,-0.000267
2022-02-25,0.000259,0.000030
2022-02-28,-0.000019,0.000240
2022-03-01,-0.000139,-0.000158


In [50]:
all_AR["AAR"] / abnormalReturns.std(axis = 1)

Date
2022-02-21   -0.005879
2022-02-22    0.003179
2022-02-23   -0.002583
2022-02-24   -0.004802
2022-02-25    0.013404
2022-02-28   -0.001291
2022-03-01   -0.007599
Freq: B, dtype: float64

In [51]:
all_AR["CAR"]/(np.sqrt(len(tickers))*abnormalReturns.std(axis = 1))

Date
2022-02-21   -0.001073
2022-02-22   -0.000936
2022-02-23    0.000158
2022-02-24   -0.001024
2022-02-25    0.000286
2022-02-28    0.002940
2022-03-01   -0.001580
Freq: B, dtype: float64

# cross_sectional

In [13]:
path_name = "L:/year2_sem3/Pembroke/supervision/Russia-Ukraine war/news"
NEWS_FILE1 = os.listdir(f"{path_name}/FT")
NEWS_FILE2 = os.listdir(f"{path_name}/REUTERS")
NEWS_FILE3 = os.listdir(f"{path_name}/TWSJ")
NEWS_FILE = [file for file in NEWS_FILE1]+[file for file in NEWS_FILE2]+[file for file in NEWS_FILE3]
NEWS_FILE

['A tragedy for Ukraine and all of Europe_2022-02-25.txt',
 'Air power counts for little in Ukraine war as caution prevails on both sides_2022-03-02.txt',
 'Baltic states appeal for Nato help after Russia’s assault on Ukraine_2022-02-24.txt',
 'Boris Johnson under fire over ‘peashooter’ Russian sanctions_2022-02-22.txt',
 'Can Europe survive without Russian gas_2022-02-23.txt',
 'CityRussia once-active market unwinds amid sanctions_2022-03-02.txt',
 'Companies rush to build cyber defences in wake of Ukraine-Russia conflict_2022-02-26.txt',
 'Concern for Ukraine is no longer enough — the west’s diplomats must act_2022-02-22.txt',
 'Conflict and capital lessons from the first world war_2022-03-02.txt',
 'Don’t mention the invasion China spins Russia’s war in Ukraine_2022-02-26.txt',
 'Efforts to supply weapons to Ukraine face multiple obstacles_2022-02-27.txt',
 'Emerging markets recoil from Ukraine war shock_2022-02-28.txt',
 'EU agrees new sanctions to hit Russian economy over Ukraine 

In [14]:
len(NEWS_FILE)

239

# Text Normalization

In [15]:
stop_words = set(stopwords.words('english'))
stop_words.discard('not')

In [16]:
def normalize(text, default_replace = ""):
    #handle URL
    text = re.sub("(http|https):\/\/\S+", default_replace, text)
    
    #remove upper capitalization
    text = text.lower()
    
    #word contraction
    for k,v in contractions.contractions_dict.items():
        text = text.replace(k,v)
     
    #tokenization
    token_list = word_tokenize(text)
    
    token_list = [token for token in token_list if token not in string.punctuation]
    token_list = [token for token in token_list if token.isalpha()]
    token_list = [token for token in token_list if not token in stop_words]

    return token_list

# Stemming

In [17]:
stemmer = SnowballStemmer("english")

In [18]:
def stem_tokens(tokens, stemmer):
    token_list = []
    for token in tokens:
        token_list.append(stemmer.stem(token))
    return token_list

In [19]:
def process_text(file):
    folder = ""
    if file in NEWS_FILE1:
        folder = "FT"
    elif file in NEWS_FILE2:
        folder = "REUTERS"
    else:
        folder = "TWSJ"
    with open(f'{path_name}/{folder}/{file}', encoding="utf8") as f:
        text = f.read()
    
    tokens = normalize(text)
    stem = stem_tokens(tokens, stemmer)

    return stem   

# Sentiment analysis 

In [20]:
positive_lexiceon = set(opinion_lexicon.positive())
negative_lexiceon = set(opinion_lexicon.negative())

In [21]:
sentiment_scores = {}
for i in range(len(NEWS_FILE)):
    date = os.path.splitext(NEWS_FILE[i])[0].split('_')[-1].split('-')[-1]
    
    cleaned_words = process_text(NEWS_FILE[i])
    if len(cleaned_words) == 0:
        print(NEWS_FILE[i])
    positive_sentiment = 0
    negative_sentiment = 0
    for word in cleaned_words:       
        if word in positive_lexiceon:
            positive_sentiment += 1
        elif word in negative_lexiceon:
            negative_sentiment += 1
        
    phi_pos = positive_sentiment / len(cleaned_words)
    phi_neg = negative_sentiment / len(cleaned_words)
        
    phi_npt = (phi_pos - phi_neg) / (phi_pos + phi_neg)
        
    sentiment_scores[NEWS_FILE[i]] = [phi_pos, phi_neg, phi_npt]

sentiment_scores

{'A tragedy for Ukraine and all of Europe_2022-02-25.txt': [0.024615384615384615,
  0.07384615384615385,
  -0.5000000000000001],
 'Air power counts for little in Ukraine war as caution prevails on both sides_2022-03-02.txt': [0.02355072463768116,
  0.059782608695652176,
  -0.43478260869565216],
 'Baltic states appeal for Nato help after Russia’s assault on Ukraine_2022-02-24.txt': [0.019867549668874173,
  0.033112582781456956,
  -0.25],
 'Boris Johnson under fire over ‘peashooter’ Russian sanctions_2022-02-22.txt': [0.0211864406779661,
  0.029661016949152543,
  -0.16666666666666669],
 'Can Europe survive without Russian gas_2022-02-23.txt': [0.02224694104560623,
  0.03114571746384872,
  -0.16666666666666666],
 'CityRussia once-active market unwinds amid sanctions_2022-03-02.txt': [0.0481283422459893,
  0.03208556149732621,
  0.19999999999999993],
 'Companies rush to build cyber defences in wake of Ukraine-Russia conflict_2022-02-26.txt': [0.01950354609929078,
  0.0797872340425532,
  -0

In [22]:
sentiment = pd.DataFrame(sentiment_scores, index = ['positive','negative','npt_positive'] )
sentiment

,A tragedy for Ukraine and all of Europe_2022-02-25.txt,Air power counts for little in Ukraine war as caution prevails on both sides_2022-03-02.txt,Baltic states appeal for Nato help after Russia’s assault on Ukraine_2022-02-24.txt,Boris Johnson under fire over ‘peashooter’ Russian sanctions_2022-02-22.txt,Can Europe survive without Russian gas_2022-02-23.txt,CityRussia once-active market unwinds amid sanctions_2022-03-02.txt,Companies rush to build cyber defences in wake of Ukraine-Russia conflict_2022-02-26.txt,Concern for Ukraine is no longer enough — the west’s diplomats must act_2022-02-22.txt,Conflict and capital lessons from the first world war_2022-03-02.txt,Don’t mention the invasion China spins Russia’s war in Ukraine_2022-02-26.txt,...,"Welcome to Batman’s Hometown. Wait, Is That Glasgow_2022-02-21.txt",What Did Biden Say as He Addressed Russia Invading Ukraine_2022-02-24.txt,What Russia’s Invasion of Ukraine Means for Investors_2022-02-24.txt,What the Russia-Ukraine Crisis Means for Businesses_2022-02-28.txt,Why Putin Is Outfoxing the West_2022-02-21.txt,Why Stocks Rebounded After Russia Invaded Ukraine_2022-02-25.txt,Will Canadian Democracy Survive Justin Trudeau_2022-02-21.txt,"With Nord Stream 2 Freeze, Germany Takes First Step to Cut Russian Gas Reliance_2022-02-23.txt","With Russia’s Invasion of Ukraine, a New Cold War Arrives_2022-02-22.txt",‘Hell’s Half-Acre’ Review Murder on the Prairie_2022-02-23.txt
positive,0.024615,0.023551,0.019868,0.021186,0.022247,0.048128,0.019504,0.020408,0.025397,0.023256,...,0.035889,0.023438,0.035714,0.019547,0.035545,0.050542,0.040179,0.013970,0.016667,0.035398
negative,0.073846,0.059783,0.033113,0.029661,0.031146,0.032086,0.079787,0.051020,0.030159,0.050740,...,0.027732,0.039062,0.050325,0.048354,0.066351,0.066787,0.066964,0.022119,0.042857,0.069027
npt_positive,-0.500000,-0.434783,-0.250000,-0.166667,-0.166667,0.200000,-0.607143,-0.428571,-0.085714,-0.371429,...,0.128205,-0.250000,-0.169811,-0.424242,-0.302326,-0.138462,-0.250000,-0.225806,-0.440000,-0.322034


In [23]:
def sentimentCount(lst,date,npt):
    if date == '21':
        lst[0] += npt
    if date == '22':
        lst[1] += npt
    elif date == '23':
        lst[2] += npt
    elif date == '24':
        lst[3] += npt
    elif date == '25':
        lst[4] += npt
    elif date == '28':
        lst[5] += npt
    elif date == '01':
        lst[6] += npt
    return lst

In [24]:
sentiment_list = [0 for i in range(7)]
for col in sentiment:
    date =  os.path.splitext(col)[0].split('_')[-1].split('-')[-1]
    sentiment_list = sentimentCount(sentiment_list,date,sentiment[col].values[2])
sentiment_list = [i/10 for i in sentiment_list]
sentiment_list

[-0.4148928641557529,
 -0.8320276419381608,
 -0.7268224587522784,
 -0.9316621735108734,
 -0.8916187926897804,
 -0.6214900225739902,
 -0.6376279648789687]

# Size

In [25]:
import pandas_datareader
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

from datetime import datetime
import matplotlib.pyplot as plt

In [26]:
size_startdate = datetime(2022,2,21)
size_enddate = datetime(2022,3,2)

# 'FTSE 100', 'NASDAQ-100', 'S&P 500', 'CAC 40', 'DAX',
#           'Nikkei 225','Hang Seng Index',  'Shanghai Composite Index', 'S&P/ASX 200', 'S&P BSE SENSEX ',
#           'S&P/TSX Composite Index', 'STI Index', 'ESTX 50 PR.EUR','Russell 2000', 'FTSE MIB Index',
#           'S&P/ASX 50', 'Nifty 50', 'OMX Stockholm 30', 'PSI 20', 'Dow Jones Industrial Average',
#           'iShares MSCI World Index ETF', 'KOSPI Composite Index', 'OMX Copenhagen 25', 'TSEC weighted index', 
#            'S&P/NZX 50 INDEX GROSS', 'STXE 600 PR.EUR','BIST 100',
#            'iShares MSCI World Small Cap UCITS ETF ', 'MSCI AC Asia Pacific EX Japan Index FAM Fund Class I EUR Accumulation ',
#             'Vanguard FTSE Developed Europe ex-U.K. Equity Index Fund GBP Acc'

tickers = ['IMOEX.ME','^FTSE','^FCHI','DAX', 'ENY=F', '^HSI', '000001.SS','^AXJO', '^BSESN', 
           '^GSPTSE', '^STI', '^STOXX50E', '^RUT', 'FTSEMIB.MI', '^AFLI', '^NSEI','^OMX',
           'PSI20.LS', '^DJI', 'XWD.TO','^KS11','^OMXC25','^TWII','^NZ50', '^STOXX',
           'XU100.IS', 'WSML.L', '0P0001GY56.F', 'IEUR',  '0P0000KSP8.L']
volumes = pdr.get_data_yahoo(tickers, start = size_startdate, end = size_enddate)["Volume"]

[*********************100%***********************]  30 of 30 completed


In [27]:
volumes

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-21,301500,0,0.0,NaN,NaN,357596100,NaN,0.0,88816100,82172,...,495000.0,215200.0,0,84864500,11238200,NaN,167740900,175222500,29685200,4487800.0
2022-02-22,326300,0,0.0,40500.0,0.0,491922500,1496500.0,0.0,110706400,574134,...,648100.0,300100.0,37023200,0,15829500,5.121900e+09,252140500,241968200,43502500,5056200.0
2022-02-23,325600,0,0.0,5300.0,0.0,416540200,1502800.0,NaN,111954900,169085,...,537700.0,200200.0,27798200,72651000,12121000,4.797430e+09,257984100,204391200,32529000,3709000.0
2022-02-24,488400,0,0.0,77200.0,0.0,871835400,2550300.0,0.0,254546900,293388,...,925900.0,458000.0,6852100,0,19586000,6.752130e+09,563236900,450927300,88823100,6003200.0
2022-02-25,377500,0,0.0,15500.0,0.0,669513500,2678300.0,0.0,209738900,115216,...,664100.0,329800.0,34948900,0,26313800,5.177060e+09,414154800,337460500,66184900,5029500.0
2022-02-28,345800,0,NaN,17600.0,0.0,760481500,3370200.0,NaN,178166400,102955,...,613300.0,404200.0,4052500,0,32268100,6.071370e+09,572922000,381363800,77052100,NaN
2022-03-01,326000,0,NaN,26300.0,0.0,824196900,2347900.0,NaN,198225900,100531,...,NaN,NaN,5684200,0,23541800,5.846230e+09,336180300,341979600,71926900,4576900.0


In [28]:
#volumes.drop(columns = ["^OMX"], inplace = True)
for col in volumes:
    if int(volumes[col].mean()) == 0:
        volumes.drop(columns = [col], inplace = True)
        continue
    for i in range(len(volumes)):
        if pd.isnull(volumes[col].iloc[i]):
            volumes[col].iloc[i] = volumes[col].iloc[i-1]
    
volumes

C:\Users\User\AppData\Local\Temp\ipykernel_5472\559055742.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volumes[col].iloc[i] = volumes[col].iloc[i-1]
C:\Users\User\AppData\Local\Temp\ipykernel_5472\559055742.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volumes[col].iloc[i] = volumes[col].iloc[i-1]
C:\Users\User\AppData\Local\Temp\ipykernel_5472\559055742.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volumes[col].iloc

,000001.SS,DAX,FTSEMIB.MI,IEUR,PSI20.LS,WSML.L,XU100.IS,XWD.TO,^AFLI,^AXJO,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-21,301500,26300.0,357596100,2347900.0,88816100,82172,2272313300,41300.0,227800,712000,...,495000.0,215200.0,0,84864500,11238200,5.846230e+09,167740900,175222500,29685200,4487800.0
2022-02-22,326300,40500.0,491922500,1496500.0,110706400,574134,2679280300,8700.0,293600,863800,...,648100.0,300100.0,37023200,0,15829500,5.121900e+09,252140500,241968200,43502500,5056200.0
2022-02-23,325600,5300.0,416540200,1502800.0,111954900,169085,2272755900,7000.0,284100,788400,...,537700.0,200200.0,27798200,72651000,12121000,4.797430e+09,257984100,204391200,32529000,3709000.0
2022-02-24,488400,77200.0,871835400,2550300.0,254546900,293388,2428056400,44500.0,388200,1109300,...,925900.0,458000.0,6852100,0,19586000,6.752130e+09,563236900,450927300,88823100,6003200.0
2022-02-25,377500,15500.0,669513500,2678300.0,209738900,115216,3298696000,9800.0,318900,888800,...,664100.0,329800.0,34948900,0,26313800,5.177060e+09,414154800,337460500,66184900,5029500.0
2022-02-28,345800,17600.0,760481500,3370200.0,178166400,102955,2813218200,54000.0,392500,1007600,...,613300.0,404200.0,4052500,0,32268100,6.071370e+09,572922000,381363800,77052100,5029500.0
2022-03-01,326000,26300.0,824196900,2347900.0,198225900,100531,3185474100,41300.0,281200,812800,...,613300.0,404200.0,5684200,0,23541800,5.846230e+09,336180300,341979600,71926900,4576900.0


In [29]:
volumes['size'] = np.log(volumes.mean(axis = 1)) / 100
volumes

,000001.SS,DAX,FTSEMIB.MI,IEUR,PSI20.LS,WSML.L,XU100.IS,XWD.TO,^AFLI,^AXJO,...,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII,size
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-21,301500,26300.0,357596100,2347900.0,88816100,82172,2272313300,41300.0,227800,712000,...,215200.0,0,84864500,11238200,5.846230e+09,167740900,175222500,29685200,4487800.0,0.199564
2022-02-22,326300,40500.0,491922500,1496500.0,110706400,574134,2679280300,8700.0,293600,863800,...,300100.0,37023200,0,15829500,5.121900e+09,252140500,241968200,43502500,5056200.0,0.200253
2022-02-23,325600,5300.0,416540200,1502800.0,111954900,169085,2272755900,7000.0,284100,788400,...,200200.0,27798200,72651000,12121000,4.797430e+09,257984100,204391200,32529000,3709000.0,0.198848
2022-02-24,488400,77200.0,871835400,2550300.0,254546900,293388,2428056400,44500.0,388200,1109300,...,458000.0,6852100,0,19586000,6.752130e+09,563236900,450927300,88823100,6003200.0,0.203336
2022-02-25,377500,15500.0,669513500,2678300.0,209738900,115216,3298696000,9800.0,318900,888800,...,329800.0,34948900,0,26313800,5.177060e+09,414154800,337460500,66184900,5029500.0,0.201728
2022-02-28,345800,17600.0,760481500,3370200.0,178166400,102955,2813218200,54000.0,392500,1007600,...,404200.0,4052500,0,32268100,6.071370e+09,572922000,381363800,77052100,5029500.0,0.202585
2022-03-01,326000,26300.0,824196900,2347900.0,198225900,100531,3185474100,41300.0,281200,812800,...,404200.0,5684200,0,23541800,5.846230e+09,336180300,341979600,71926900,4576900.0,0.201701


# risk

In [30]:
risk_startdate = datetime(2022,2,3)
risk_enddate = datetime(2022,3,2)
riskPrices = pdr.get_data_yahoo(tickers, start = risk_startdate, end = risk_enddate)["Adj Close"]
riskPrices

[*********************100%***********************]  30 of 30 completed


,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-03,NaN,33002.828125,NaN,29.737644,26950.0,27089.0,53.512112,3471.179932,5579.149902,6.9040,...,2707.820068,17560.199219,12335.320312,2277.669922,1768.459961,1991.030029,3315.989990,468.630005,4141.020020,NaN
2022-02-04,NaN,32799.921875,NaN,29.557129,27290.0,26604.0,53.341206,3471.010010,5605.879883,6.8170,...,2750.260010,17516.300781,12279.559570,2254.300049,1723.250000,2002.359985,3331.409912,462.149994,4086.580078,NaN
2022-02-07,3429.580078,33026.828125,121.019997,29.566631,27200.0,26329.0,53.417164,3470.780029,5605.339844,6.8740,...,2745.060059,17213.599609,NaN,2278.639893,1735.719971,2012.599976,3366.479980,465.279999,4120.560059,17900.300781
2022-02-08,3452.629883,32876.410156,121.334999,29.718645,27400.0,26412.0,53.569077,3551.649902,5643.490234,6.9175,...,2746.469971,17266.750000,12316.440430,2274.399902,1698.859985,2045.369995,3401.739990,465.339996,4129.250000,17966.560547
2022-02-09,3479.949951,33566.808594,123.418999,30.186085,27830.0,27129.0,54.471077,3638.929932,5696.589844,7.0525,...,2768.850098,17463.800781,12467.049805,2313.139893,1765.050049,2083.500000,3420.040039,473.329987,4204.089844,18151.759766
2022-02-10,3485.909912,33341.570312,124.013000,29.908661,27390.0,27190.0,53.825432,3655.760010,5671.700195,7.0955,...,2771.929932,17605.849609,12413.049805,2314.260010,1755.060059,2051.159912,3428.000000,472.350006,4197.069824,18338.050781
2022-02-11,3462.949951,32924.539062,123.558998,29.281605,26930.0,26966.0,52.819000,3546.620117,5590.859863,7.0060,...,2747.709961,17374.750000,12173.780273,2293.770020,1733.739990,2030.150024,3428.949951,469.570007,4155.229980,18310.939453
2022-02-14,3428.879883,32244.550781,122.719002,28.939575,27060.0,26415.0,52.277805,3481.379883,5507.660156,6.9170,...,2704.479980,16842.800781,11950.139648,2240.969971,1692.869995,2020.790039,3421.199951,460.959991,4064.449951,17997.669922
2022-02-15,3446.090088,32898.000000,122.588997,29.775650,27450.0,26968.0,53.436153,3600.290039,5643.620117,6.9870,...,2676.540039,17352.449219,11938.320312,2282.060059,1742.140015,2076.459961,3421.379883,467.559998,4143.709961,17951.810547


In [31]:
riskReturns  = riskPrices.pct_change(1).dropna()
#marketReturns = np.where(int(marketReturns) == 0, )
riskReturns

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-08,0.006721,-0.004554,0.002603,0.005141,0.007353,0.003152,0.002844,0.023300,0.006806,0.006328,...,0.000514,0.003088,0.003003,-0.001861,-0.021236,0.016282,0.010474,0.000129,0.002109,0.003702
2022-02-09,0.007913,0.021000,0.017176,0.015729,0.015693,0.027147,0.016838,0.024575,0.009409,0.019516,...,0.008149,0.011412,0.012228,0.017033,0.038961,0.018642,0.005380,0.017170,0.018124,0.010308
2022-02-10,0.001713,-0.006710,0.004813,-0.009190,-0.015810,0.002249,-0.011853,0.004625,-0.004369,0.006097,...,0.001112,0.008134,-0.004331,0.000484,-0.005660,-0.015522,0.002327,-0.002070,-0.001670,0.010263
2022-02-11,-0.006587,-0.012508,-0.003661,-0.020966,-0.016794,-0.008238,-0.018698,-0.029854,-0.014253,-0.012614,...,-0.008738,-0.013126,-0.019276,-0.008854,-0.012148,-0.010243,0.000277,-0.005885,-0.009969,-0.001478
2022-02-14,-0.009838,-0.020653,-0.006798,-0.011681,0.004827,-0.020433,-0.010246,-0.018395,-0.014881,-0.012703,...,-0.015733,-0.030616,-0.018371,-0.023019,-0.023573,-0.004610,-0.002260,-0.018336,-0.021847,-0.017108
2022-02-15,0.005019,0.020265,-0.001059,0.028890,0.014412,0.020935,0.022158,0.034156,0.024686,0.010120,...,-0.010331,0.030259,-0.000989,0.018336,0.029104,0.027549,0.000053,0.014318,0.019501,-0.002548
2022-02-16,0.005728,-0.003451,0.011763,0.005424,0.000364,0.000037,0.004264,0.012810,0.003413,0.004007,...,0.019854,-0.001743,0.020842,-0.008173,-0.002267,0.001373,0.005238,0.000449,-0.001566,0.015578
2022-02-17,0.000638,-0.009781,0.003265,-0.020248,-0.021122,-0.011124,-0.017339,-0.037113,-0.000766,-0.010192,...,0.005279,-0.001016,0.005718,-0.015136,-0.010793,-0.024633,0.000660,-0.006884,-0.005808,0.002035
2022-02-18,0.006551,-0.009225,-0.007217,-0.010106,-0.001116,-0.006074,-0.006482,-0.033542,-0.004655,-0.013898,...,0.000157,-0.001635,-0.009377,-0.014100,-0.020076,-0.009250,-0.003682,-0.008051,-0.009460,-0.001983


In [32]:
risk_dict = {}
for col in riskReturns:
    risk_dict[col] = []
    for i in range(9,len(riskReturns)):
        risk_dict[col].append(np.std(riskReturns[col].iloc[i-9 : i], ddof = 1))
risk_dict

{'000001.SS': [0.0062927600319262166,
  0.006055786402031428,
  0.006439834499736097,
  0.007236445344492748,
  0.009041465614570197,
  0.00866431815042508,
  0.008571716112866744],
 '0P0000KSP8.L': [0.01423110972984484,
  0.0151778620576161,
  0.012196635161151737,
  0.012316242776536925,
  0.014295733424939183,
  0.018714039674803146,
  0.016816402124777294],
 '0P0001GY56.F': [0.008203092298075622,
  0.008420898469952583,
  0.007632488055945492,
  0.007496779798525365,
  0.009113865177722877,
  0.009030226878672514,
  0.009066602548044655],
 'DAX': [0.016875799328961546,
  0.01669408634684393,
  0.017118427126135084,
  0.01720916905650578,
  0.017099565846572604,
  0.021447039374525793,
  0.0198866616831784],
 'ENY=F': [0.013667362205106328,
  0.013296520593558198,
  0.011519912866948444,
  0.011157309210169617,
  0.010129401969949938,
  0.014216260763612452,
  0.013541889860651883],
 'FTSEMIB.MI': [0.015123404433028367,
  0.01622306780587528,
  0.012287216858635285,
  0.012043394155

In [33]:
risks = pd.DataFrame(risk_dict)
risks

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
0,0.006293,0.014231,0.008203,0.016876,0.013667,0.015123,0.014579,0.027814,0.012342,0.012211,...,0.010727,0.016716,0.013370,0.014118,0.022335,0.017439,0.004370,0.011057,0.013257,0.009606
1,0.006056,0.015178,0.008421,0.016694,0.013297,0.016223,0.014490,0.042271,0.014606,0.012477,...,0.010726,0.016748,0.013312,0.014778,0.021803,0.016343,0.003046,0.011735,0.014810,0.009623
2,0.006440,0.012197,0.007632,0.017118,0.011520,0.012287,0.013062,0.041294,0.013861,0.009883,...,0.011076,0.016249,0.012297,0.012363,0.015597,0.014810,0.004495,0.009323,0.012502,0.010241
3,0.007236,0.012316,0.007497,0.017209,0.011157,0.012043,0.012909,0.041013,0.013872,0.009314,...,0.011284,0.015786,0.012383,0.011991,0.015595,0.015063,0.004376,0.009304,0.012453,0.009629
4,0.009041,0.014296,0.009114,0.017100,0.010129,0.017150,0.012915,0.113067,0.013958,0.010722,...,0.013758,0.021616,0.011560,0.013955,0.015779,0.018382,0.011876,0.013316,0.016064,0.012484
5,0.008664,0.018714,0.009030,0.021447,0.014216,0.021986,0.017151,0.141123,0.016962,0.017260,...,0.013742,0.022073,0.009843,0.018714,0.018598,0.020002,0.012382,0.018200,0.021259,0.011705
6,0.008572,0.016816,0.009067,0.019887,0.013542,0.020345,0.015999,0.139717,0.015105,0.017988,...,0.013578,0.019202,0.010983,0.016628,0.018795,0.017505,0.012839,0.017210,0.019732,0.011724


In [34]:
risks.index = volumes.index
risks

,000001.SS,0P0000KSP8.L,0P0001GY56.F,DAX,ENY=F,FTSEMIB.MI,IEUR,IMOEX.ME,PSI20.LS,WSML.L,...,^KS11,^NSEI,^NZ50,^OMX,^OMXC25,^RUT,^STI,^STOXX,^STOXX50E,^TWII
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-21,0.006293,0.014231,0.008203,0.016876,0.013667,0.015123,0.014579,0.027814,0.012342,0.012211,...,0.010727,0.016716,0.013370,0.014118,0.022335,0.017439,0.004370,0.011057,0.013257,0.009606
2022-02-22,0.006056,0.015178,0.008421,0.016694,0.013297,0.016223,0.014490,0.042271,0.014606,0.012477,...,0.010726,0.016748,0.013312,0.014778,0.021803,0.016343,0.003046,0.011735,0.014810,0.009623
2022-02-23,0.006440,0.012197,0.007632,0.017118,0.011520,0.012287,0.013062,0.041294,0.013861,0.009883,...,0.011076,0.016249,0.012297,0.012363,0.015597,0.014810,0.004495,0.009323,0.012502,0.010241
2022-02-24,0.007236,0.012316,0.007497,0.017209,0.011157,0.012043,0.012909,0.041013,0.013872,0.009314,...,0.011284,0.015786,0.012383,0.011991,0.015595,0.015063,0.004376,0.009304,0.012453,0.009629
2022-02-25,0.009041,0.014296,0.009114,0.017100,0.010129,0.017150,0.012915,0.113067,0.013958,0.010722,...,0.013758,0.021616,0.011560,0.013955,0.015779,0.018382,0.011876,0.013316,0.016064,0.012484
2022-02-28,0.008664,0.018714,0.009030,0.021447,0.014216,0.021986,0.017151,0.141123,0.016962,0.017260,...,0.013742,0.022073,0.009843,0.018714,0.018598,0.020002,0.012382,0.018200,0.021259,0.011705
2022-03-01,0.008572,0.016816,0.009067,0.019887,0.013542,0.020345,0.015999,0.139717,0.015105,0.017988,...,0.013578,0.019202,0.010983,0.016628,0.018795,0.017505,0.012839,0.017210,0.019732,0.011724


In [35]:
reg =pd.DataFrame(risks.mean(axis = 1), columns = ['risk'])
reg

,risk
Date,
2022-02-21,0.012932
2022-02-22,0.013447
2022-02-23,0.012213
2022-02-24,0.012174
2022-02-25,0.017480
2022-02-28,0.020865
2022-03-01,0.020029


In [36]:
reg['sentiment'] = sentiment_list
reg

,risk,sentiment
Date,,
2022-02-21,0.012932,-0.414893
2022-02-22,0.013447,-0.832028
2022-02-23,0.012213,-0.726822
2022-02-24,0.012174,-0.931662
2022-02-25,0.017480,-0.891619
2022-02-28,0.020865,-0.621490
2022-03-01,0.020029,-0.637628


In [37]:
reg['size'] = volumes['size']
reg

,risk,sentiment,size
Date,,,
2022-02-21,0.012932,-0.414893,0.199564
2022-02-22,0.013447,-0.832028,0.200253
2022-02-23,0.012213,-0.726822,0.198848
2022-02-24,0.012174,-0.931662,0.203336
2022-02-25,0.017480,-0.891619,0.201728
2022-02-28,0.020865,-0.621490,0.202585
2022-03-01,0.020029,-0.637628,0.201701


In [38]:
reg ['AAR'] = all_AR['AAR']
reg

,risk,sentiment,size,AAR
Date,,,,
2022-02-21,0.012932,-0.414893,0.199564,-0.000134
2022-02-22,0.013447,-0.832028,0.200253,0.000051
2022-02-23,0.012213,-0.726822,0.198848,-0.000038
2022-02-24,0.012174,-0.931662,0.203336,-0.000229
2022-02-25,0.017480,-0.891619,0.201728,0.000259
2022-02-28,0.020865,-0.621490,0.202585,-0.000019
2022-03-01,0.020029,-0.637628,0.201701,-0.000139


In [39]:
reg1 = reg.iloc[0:2]
reg2 = reg.iloc[1:3]
reg3 = reg.iloc[2:4]
reg4 = reg.iloc[3:5]
reg5 = reg.iloc[4:6]
reg6 = reg.iloc[5:7]

In [40]:
import statsmodels.formula.api as smf

In [41]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       0.588
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     1.430
Date:                Sat, 12 Aug 2023   Prob (F-statistic):              0.388
Time:                        10:14:05   Log-Likelihood:                 54.929
No. Observations:                   7   AIC:                            -101.9
Df Residuals:                       3   BIC:                            -102.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0157      0.010      1.612      0.2

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 7 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [42]:
# Conduct the Breusch-Pagan test
names = ['Lagrange multiplier statistic', 'p-value',
         'f-value', 'f p-value']
 
# Get the test result
test_result = sms.het_breuschpagan(fit.resid, fit.model.exog)
 
lzip(names, test_result)

[('Lagrange multiplier statistic', 1.9924087586633297),
 ('p-value', 0.5739837892020851),
 ('f-value', 0.39787767464252904),
 ('f p-value', 0.7654850123549278)]

In [43]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg1).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                nan
Time:                        10:14:05   Log-Likelihood:                 87.518
No. Observations:                   2   AIC:                            -171.0
Df Residuals:                       0   BIC:                            -173.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0003        inf         -0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [44]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg2).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                nan
Time:                        10:14:05   Log-Likelihood:                 84.512
No. Observations:                   2   AIC:                            -165.0
Df Residuals:                       0   BIC:                            -167.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0006        inf         -0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [45]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg3).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                nan
Time:                        10:14:05   Log-Likelihood:                 85.074
No. Observations:                   2   AIC:                            -166.1
Df Residuals:                       0   BIC:                            -168.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0006        inf          0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [46]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg4).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                nan
Time:                        10:14:05   Log-Likelihood:                 79.426
No. Observations:                   2   AIC:                            -154.9
Df Residuals:                       0   BIC:                            -157.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0105        inf          0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [47]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg5).fit()
print(fit.summary())

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                nan
Time:                        10:14:05   Log-Likelihood:                 84.890
No. Observations:                   2   AIC:                            -165.8
Df Residuals:                       0   BIC:                            -168.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0006        inf         -0        n

In [48]:
# fit regression model
fit = smf.ols('AAR ~ risk+sentiment+size', data=reg6).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 12 Aug 2023   Prob (F-statistic):                nan
Time:                        10:14:05   Log-Likelihood:                 79.696
No. Observations:                   2   AIC:                            -155.4
Df Residuals:                       0   BIC:                            -158.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0042        inf          0        n

C:\Users\User\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 2 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1749: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1671: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
